In [5]:

# if kernel crashes, make sure pywin32 and pipywin32 are installed. 
# Followed instructions here: https://github.com/jupyter/notebook/issues/4909 
import win32api
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import microdf as mdf
import os
import us



In [6]:
# Import data from Ipums
person_raw = pd.read_csv("cps_00041.csv.gz")

# Create copy and lower column names
person = person_raw.copy(deep=True)
person.columns = person.columns.str.lower()
person.asecwt /= 3

# Create booleans for demographics
person["adult"] = person.age > 17
person["child"] = person.age < 18

person["black"] = person.race == 200
person["white_non_hispanic"] = (person.race == 100) & (person.hispan == 0)
person["hispanic"] = (person.hispan > 1) & person.hispan < 700
person["pwd"] = person.diffany == 2
person["non_citizen"] = person.citizen == 5
person["non_citizen_child"] = (person.citizen == 5) & (person.age < 18)
person["non_citizen_adult"] = (person.citizen == 5) & (person.age > 17)

# Remove NIUs
person["taxinc"].replace({9999999: 0}, inplace=True)
person["adjginc"].replace({99999999: 0}, inplace=True)
person["incss"].replace({999999: 0}, inplace=True)
person["incssi"].replace({999999: 0}, inplace=True)
person["incunemp"].replace({99999: 0}, inplace=True)
person["incunemp"].replace({999999: 0}, inplace=True)
person["ctccrd"].replace({999999: 0}, inplace=True)
person["actccrd"].replace({99999: 0}, inplace=True)
person["eitcred"].replace({9999: 0}, inplace=True)
person["fica"].replace({99999: 0}, inplace=True)
person["fedtaxac"].replace({99999999: 0}, inplace=True)
person["stataxac"].replace({9999999: 0}, inplace=True)

# Change fip codes to state names
# Change fip codes to state names
person["state"] = (
    person["statefip"].astype(str)
    # pad leading zero or wrong number of states
    .apply("{:0>2}".format)
    # lookup full state name from fips code
    .apply(lambda x: us.states.lookup(x))
    # change us package formatting to string
    .astype(str)
)
# drop original statefip column from dataframe
person = person.drop(columns=["statefip"])

# Aggregate deductible and refundable child tax credits
person["ctc"] = person.ctccrd + person.actccrd

# Calculate the number of people per smp unit
person["person"] = 1
spm = person.groupby(["spmfamunit", "year"])[["person"]].sum()
spm.columns = ["numper"]
person = person.merge(spm, left_on=["spmfamunit", "year"], right_index=True)

person["weighted_state_tax"] = person.asecwt * person.stataxac
person["weighted_agi"] = person.asecwt * person.adjginc

# Calculate the total taxable income and total people in each state
state_groups_taxinc = person.groupby(["state"])[
    ["weighted_state_tax", "weighted_agi"]
].sum()
state_groups_taxinc.columns = ["state_tax_revenue", "state_taxable_income"]
person = person.merge(
    state_groups_taxinc, left_on=["state"], right_index=True
)

person.head().T

,0,1,2,3,4
year,2018,2018,2018,2018,2018
asecwt,438.673333,274.763333,274.763333,243.89,368.823333
age,73,48,55,12,6
race,100,100,100,100,200
citizen,1,1,1,1,1
hispan,0,0,0,0,0
diffany,2,1,1,0,0
incss,6000,0,0,0,0
incssi,0,0,0,0,0
incunemp,0,0,0,0,0


In [7]:
# Create dataframe with aggregated spm unit data
PERSON_COLUMNS = [
    "adjginc",
    "fica",
    "fedtaxac",
    "ctc",
    "incssi",
    "incunemp",
    "eitcred",
    "child",
    "adult",
    "non_citizen",
    "non_citizen_child",
    "non_citizen_adult",
    "person",
    "stataxac",
]
SPMU_COLUMNS = [
    "spmheat",
    "spmsnap",
    "spmfamunit",
    "spmthresh",
    "spmtotres",
    "spmwt",
    "year",
    "state",
    "state_tax_revenue",
    "state_taxable_income",
]

spmu = person.groupby(SPMU_COLUMNS, observed=False)[PERSON_COLUMNS].sum().reset_index()
spmu[["fica", "fedtaxac", "stataxac"]] *= -1
spmu.rename(columns={"person": "numper"}, inplace=True)

spmu.spmwt /= 3

spmu.head()

,spmheat,spmsnap,spmfamunit,spmthresh,spmtotres,spmwt,year,state,state_tax_revenue,state_taxable_income,...,incssi,incunemp,eitcred,child,adult,non_citizen,non_citizen_child,non_citizen_adult,numper,stataxac
0,0.0,0,1001,14700.00000,86459.000,517.633333,2020,Maine,1.526793e+09,4.017900e+10,...,0,0,0,0,2,0,0,0,2,-4624
1,0.0,0,2001,11004.31484,47514.000,438.673333,2018,Maine,1.526793e+09,4.017900e+10,...,0,0,0,0,1,0,0,0,1,0
2,0.0,0,2001,14700.00000,55275.000,330.163333,2020,Maine,1.526793e+09,4.017900e+10,...,0,0,0,0,2,0,0,0,2,0
3,0.0,0,3001,11920.00000,28653.000,501.756667,2020,Maine,1.526793e+09,4.017900e+10,...,0,0,0,0,1,0,0,0,1,-1152
4,0.0,0,4001,26449.08415,105232.008,274.763333,2018,Maine,1.526793e+09,4.017900e+10,...,0,0,0,3,2,0,0,0,5,-4091


In [8]:
spmu.describe().T

,count,mean,std,min,25%,50%,75%,max
spmheat,205618.0,1.503946e+01,1.174539e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e+03
spmsnap,205618.0,2.303782e+02,1.036016e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+04
spmfamunit,205618.0,4.730608e+07,2.695320e+07,1.001000e+03,2.393125e+07,4.742550e+07,7.104775e+07,9.463300e+07
spmthresh,205618.0,2.084751e+04,8.769906e+03,9.075782e+03,1.392000e+04,1.835654e+04,2.618000e+04,9.315053e+04
spmtotres,205618.0,6.334466e+04,6.793154e+04,-9.898173e+05,2.433325e+04,4.635250e+04,8.102476e+04,1.534490e+06
spmwt,205618.0,6.563984e+02,4.049073e+02,3.108000e+01,3.192058e+02,6.170533e+02,9.224800e+02,3.866477e+03
year,205618.0,2.018961e+03,8.066634e-01,2.018000e+03,2.018000e+03,2.019000e+03,2.020000e+03,2.020000e+03
state_tax_revenue,205618.0,1.200535e+10,1.790241e+10,0.000000e+00,1.365168e+09,4.514997e+09,1.284825e+10,6.109081e+10
state_taxable_income,205618.0,3.791404e+11,4.063929e+11,1.739814e+10,7.883665e+10,2.256971e+11,4.656021e+11,1.401512e+12
adjginc,205618.0,8.133351e+04,1.067784e+05,-9.999000e+03,1.985025e+04,5.552000e+04,1.081548e+05,2.979999e+06


In [9]:
person.describe().T

,count,mean,std,min,25%,50%,75%,max
year,518144.0,2.018957e+03,8.065911e-01,2.018000e+03,2.018000e+03,2.019000e+03,2.020000e+03,2.020000e+03
asecwt,518144.0,6.258107e+02,3.892790e+02,2.985000e+01,3.106567e+02,5.975833e+02,8.297133e+02,4.669547e+03
age,518144.0,3.741924e+01,2.297673e+01,0.000000e+00,1.700000e+01,3.700000e+01,5.600000e+01,8.500000e+01
race,518144.0,1.718125e+02,1.758615e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,8.300000e+02
citizen,518144.0,1.499572e+00,1.225419e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
hispan,518144.0,4.599312e+01,1.294363e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.120000e+02
diffany,518144.0,8.705244e-01,5.395108e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
incss,518144.0,2.357574e+03,6.389099e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e+04
incssi,518144.0,1.472852e+02,1.287307e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+04
incunemp,518144.0,4.987590e+01,7.381988e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.860000e+04


In [10]:
spmu.head().T

,0,1,2,3,4
spmheat,0.0,0.0,0.0,0.0,0.0
spmsnap,0,0,0,0,0
spmfamunit,1001,2001,2001,3001,4001
spmthresh,14700.0,11004.31484,14700.0,11920.0,26449.08415
spmtotres,86459.0,47514.0,55275.0,28653.0,105232.008
spmwt,517.633333,438.673333,330.163333,501.756667,274.763333
year,2020,2018,2020,2020,2018
state,Maine,Maine,Maine,Maine,Maine
state_tax_revenue,1526793245.48,1526793245.48,1526793245.48,1526793245.48,1526793245.48
state_taxable_income,40179001414.286758,40179001414.286758,40179001414.286758,40179001414.286758,40179001414.286758


In [11]:
spmu.state.nunique()

51

# calculate US stuff first

In [12]:
person.head().T

,0,1,2,3,4
year,2018,2018,2018,2018,2018
asecwt,438.673333,274.763333,274.763333,243.89,368.823333
age,73,48,55,12,6
race,100,100,100,100,200
citizen,1,1,1,1,1
hispan,0,0,0,0,0
diffany,2,1,1,0,0
incss,6000,0,0,0,0
incssi,0,0,0,0,0
incunemp,0,0,0,0,0


# Calculate Poverty Rates

In [13]:
# create boolean column for individual's poverty status, 1=poor
person["poor"] = person.spmthresh > person.spmtotres

# create a column for all selected demographic variables
# that will be used to calculate poverty rates
demog_cols = [
    "person",
    "adult",
    "child",
    "black",
    "white_non_hispanic",
    "hispanic",
    "pwd",
    "non_citizen",
    "non_citizen_adult",
    "non_citizen_child",
]

poor_pop=person.loc[person.poor]

# calculate weighted sum of people living in poverty
mdf.weighted_sum(poor_pop,demog_cols,'asecwt')
# calculate poverty RATE for each DEMOGRAPHIC in US
pov_rate_us = mdf.weighted_sum(poor_pop, demog_cols, "asecwt") / mdf.weighted_sum(
    person, demog_cols, w="asecwt"
)
# add name to series
pov_rate_us.name = "US"
# calculate poverty RATE for each group by state
pov_rates = mdf.weighted_sum(
    poor_pop, demog_cols, "asecwt", groupby="state"
) / mdf.weighted_sum(person, demog_cols, w="asecwt", groupby="state")

# append US statistics as additional 'state'
pov_df = pov_rates.append(pov_rate_us)

# melt df from wide to long format
pov_df = pov_df.melt(ignore_index=False, var_name="demog")

pov_df.insert(loc=1, column="metric", value="pov_rate")
pov_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 520 entries, Alabama to US
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   demog   520 non-null    object 
 1   metric  520 non-null    object 
 2   value   520 non-null    float64
dtypes: float64(1), object(2)
memory usage: 16.2+ KB


In [14]:
pov_df.head()

,demog,metric,value
state,,,
Alabama,person,pov_rate,0.127468
Alaska,person,pov_rate,0.132965
Arizona,person,pov_rate,0.119701
Arkansas,person,pov_rate,0.130987
California,person,pov_rate,0.174174


# Calculate Population

In [15]:
# calculate POPULATION for each DEMOGRAPHIC in US
pop_us = mdf.weighted_sum(
    person, demog_cols, w="asecwt"
)
# add name to series
pop_us.name = "US"
# calculate POPULATION for each group by state
pop_states = mdf.weighted_sum(person, demog_cols, w="asecwt", groupby="state")

# append US statistics as additional 'state'
pop_df = pop_states.append(pop_us)

# melt df from wide to long format
pop_df = pop_df.melt(ignore_index=False, var_name="demog")

pop_df.insert(loc=1, column="metric", value="pop")
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 520 entries, Alabama to US
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   demog   520 non-null    object 
 1   metric  520 non-null    object 
 2   value   520 non-null    float64
dtypes: float64(1), object(2)
memory usage: 16.2+ KB


In [16]:
pop_df

,demog,metric,value
state,,,
Alabama,person,pop,4.858162e+06
Alaska,person,pop,7.147047e+05
Arizona,person,pop,7.186145e+06
Arkansas,person,pop,2.922687e+06
California,person,pop,3.931580e+07
...,...,...,...
Washington,non_citizen_child,pop,5.411946e+04
West Virginia,non_citizen_child,pop,1.951943e+03
Wisconsin,non_citizen_child,pop,1.017909e+04


# calculate gini

In [22]:
person.head().T

,0,1,2,3,4
year,2018,2018,2018,2018,2018
asecwt,438.673333,274.763333,274.763333,243.89,368.823333
age,73,48,55,12,6
race,100,100,100,100,200
citizen,1,1,1,1,1
hispan,0,0,0,0,0
diffany,2,1,1,0,0
incss,6000,0,0,0,0
incssi,0,0,0,0,0
incunemp,0,0,0,0,0


In [40]:
# Caluclate original gini
person["spm_resources_per_person"] = (
    person.spmtotres / person.numper
)
# Caluclate original gini for US
gini_us = pd.Series(mdf.gini(person,"spm_resources_per_person", "asecwt"))
# add name to series
gini_us.index = ["US"]


# calculate gini for each group by state
gini_states = mdf.gini(person,"spm_resources_per_person", w="asecwt", groupby="state")

# append US statistics as additional 'state'
gini_ser = gini_states.append(gini_us)
gini_ser.name="gini"

# Calculate the original poverty gap
# TODO preprocess
spmu["poverty_gap"] = np.where(
    spmu.spmtotres < spmu.spmthresh,
    spmu.spmthresh - spmu.spmtotres,
    0,
)
# TODO preprocess for original_poverty_gap
poverty_gap_us = pd.Series(mdf.weighted_sum(spmu, "poverty_gap", w="spmwt"))


# add name to series
poverty_gap_us.index = ["US"]


# calculate gini for each group by state
poverty_gap_states = mdf.weighted_sum(spmu, "poverty_gap", w="spmwt", groupby='state')

# append US statistics as additional 'state'
poverty_gap_ser = poverty_gap_states.append(poverty_gap_us)
poverty_gap_ser.name="poverty_gap"

poverty_gap_ser.to_frame().merge(gini_ser.to_frame(),left_index=True,right_index=True)

,poverty_gap,gini
Alabama,2.356088e+09,0.446140
Alaska,4.940523e+08,0.450744
Arizona,3.014411e+09,0.439236
Arkansas,1.429620e+09,0.439355
California,2.973458e+10,0.460103
Colorado,3.468975e+09,0.438656
Connecticut,1.490145e+09,0.421393
Delaware,4.680860e+08,0.429479
District of Columbia,6.103180e+08,0.474598
Florida,1.419641e+10,0.456744


In [37]:
gini_ser.to_frame()

,gini
Alabama,0.446140
Alaska,0.450744
Arizona,0.439236
Arkansas,0.439355
California,0.460103
Colorado,0.438656
Connecticut,0.421393
Delaware,0.429479
District of Columbia,0.474598
Florida,0.456744


# Nothing Important After This

In [34]:
spmu.head().T

,0,1,2,3,4
spmheat,0.0,0.0,0.0,0.0,0.0
spmsnap,0,0,0,0,0
spmfamunit,1001,2001,2001,3001,4001
spmthresh,14700.0,11004.31484,14700.0,11920.0,26449.08415
spmtotres,86459.0,47514.0,55275.0,28653.0,105232.008
spmwt,517.633333,438.673333,330.163333,501.756667,274.763333
year,2020,2018,2020,2020,2018
state,Maine,Maine,Maine,Maine,Maine
state_tax_revenue,1526793245.48,1526793245.48,1526793245.48,1526793245.48,1526793245.48
state_taxable_income,40179001414.286758,40179001414.286758,40179001414.286758,40179001414.286758,40179001414.286758


In [32]:
gini_us

0    0.44605
Name: US, dtype: float64

In [18]:
# calculate gini for each group by state
gini_states = mdf.gini(person, w="asecwt", groupby="state")

# append US statistics as additional 'state'
gini_df = gini_states.append(gini_us)

# melt df from wide to long format
gini_df = gini_df.melt(ignore_index=False, var_name="demog")

52

In [19]:
pov_df['metric']='pov_rate'
pov_df.head()

,demog,metric,value
state,,,
Alabama,person,pov_rate,0.127468
Alaska,person,pov_rate,0.132965
Arizona,person,pov_rate,0.119701
Arkansas,person,pov_rate,0.130987
California,person,pov_rate,0.174174


Next Steps:
We want to have these orignal poverty rates + the total population
- Could be done best as a long dataframe, just filtered

In [20]:
# person[demog_cols]*person['poor']
# person.poor.shape
person[demog_cols] & person.poor

MemoryError: Unable to allocate 1.95 TiB for an array with shape (518144, 518144) and data type float64

We're having that weird problem again. Issue isn't groupby

In [ ]:
population = person.asecwt.sum()
adult_pop = (person.asecwt * person.adult).sum()
child_pop = (person.asecwt * person.child).sum()
black_pop = (person.asecwt * person.black).sum()
white_non_hispanic_pop = (person.asecwt * person.white_non_hispanic).sum()
hispanic_pop = (person.asecwt * person.hispanic).sum()
pwd_pop = (person.asecwt * person.pwd).sum()
non_citizen_pop = (person.asecwt * person.non_citizen).sum()
non_citizen_adult_pop = (person.asecwt * person.non_citizen_adult).sum()
non_citizen_child_pop = (person.asecwt * person.non_citizen_child).sum()

# whole us population
us_pops = mdf.weighted_sum(
    df=person,
    col=[
        "adult",
        "child",
        "black",
        "white_non_hispanic",
        "hispanic",
        "pwd",
        "non_citizen",
        "non_citizen_adult",
        "non_citizen_child",
    ],
    w="asecwt",
)


adult                 2.506244e+08
child                 7.363562e+07
black                 4.283329e+07
white_non_hispanic    1.951494e+08
hispanic              3.242600e+08
pwd                   3.043038e+07
non_citizen           2.307845e+07
non_citizen_adult     2.109232e+07
non_citizen_child     1.986129e+06
dtype: float64

In [ ]:
# each state population
state_pops = mdf.weighted_sum(
    df=person,
    col=[
        "adult",
        "child",
        "black",
        "white_non_hispanic",
        "hispanic",
        "pwd",
        "non_citizen",
        "non_citizen_adult",
        "non_citizen_child",
    ],
    w="asecwt",
    groupby=['state']
)
state_pops.head()

,adult,child,black,white_non_hispanic,hispanic,pwd,non_citizen,non_citizen_adult,non_citizen_child
statefip,,,,,,,,,
Delaware,7.740508e+05,1.992660e+05,2.233194e+05,5.962714e+05,9.733168e+05,9.249450e+04,5.218614e+04,4.866105e+04,3525.093333
District of Columbia,5.679748e+05,1.286088e+05,3.232205e+05,2.672579e+05,6.965836e+05,5.750852e+04,6.035891e+04,5.500731e+04,5351.603333
Florida,1.689726e+07,4.282094e+06,3.528064e+06,1.129305e+07,2.117935e+07,1.966757e+06,2.102128e+06,1.835257e+06,266870.983333
Georgia,7.853768e+06,2.510106e+06,3.374832e+06,5.217768e+06,1.036387e+07,8.826785e+05,6.746638e+05,6.129787e+05,61685.106667
Hawaii,1.085784e+06,3.044653e+05,2.428359e+04,2.730273e+05,1.390249e+06,1.215873e+05,9.697518e+04,8.698362e+04,9991.560000


Ok, so what's a dumb way and a smart way to get the poverty rates?

We need to use spm for that one, I guess.

Or no, we can still just use spm thresholds.

We should calculate is_poor, which will be a column in the dataframe

Then 